# Sesión 12: DISTRIBUCIONES DE PROBABILIDAD A PRIORI Y POSTERIOR
Realizado por:

**- Ruelas Flores, César Diego**

## VARIABLES GLOBALES

In [1]:
# %pip install polars numpy matplotlib scikit-learn category_encoders

In [2]:
# %pip install pyarrow

In [3]:
import sys
!{sys.executable} -m pip install pyarrow


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Instalación de librerías necesarias
# !pip install polars scikit-learn category-encoders pytest

import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from category_encoders import OneHotEncoder

# Semilla global para reproducibilidad
SEED = 42

# Ruta al archivo de datos
DATA_PATH = "obesidad.csv"


## FUNCIONES

In [5]:
%%writefile utils.py

import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from category_encoders import OneHotEncoder
from typing import Tuple, List

def load_data(path: str) -> pl.DataFrame:
    """
    Carga el dataset desde un archivo CSV utilizando polars.

    Args:
        path (str): Ruta al archivo CSV.

    Returns:
        pl.DataFrame: DataFrame con los datos cargados.
    """
    return pl.read_csv(path)

def check_missing_values(df: pl.DataFrame) -> pl.DataFrame:
    """
    Verifica la presencia de valores faltantes en el DataFrame.

    Args:
        df (pl.DataFrame): DataFrame a verificar.

    Returns:
        pl.DataFrame: DataFrame con el conteo de valores faltantes por columna.
    """
    return df.null_count()

def remove_outliers(df: pl.DataFrame, columns: List[str]) -> pl.DataFrame:
    """
    Elimina outliers utilizando el método del rango intercuartílico (IQR).

    Args:
        df (pl.DataFrame): DataFrame original.
        columns (List[str]): Lista de nombres de columnas numéricas.

    Returns:
        pl.DataFrame: DataFrame sin outliers.
    """
    for col in columns:
        q1 = df.select(pl.col(col).quantile(0.25)).to_series()[0]
        q3 = df.select(pl.col(col).quantile(0.75)).to_series()[0]
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df.filter((pl.col(col) >= lower_bound) & (pl.col(col) <= upper_bound))
    return df

def encode_categorical(df: pl.DataFrame, categorical_cols: List[str]) -> Tuple[pl.DataFrame, List[str]]:
    """
    Codifica variables categóricas utilizando one-hot encoding con Polars.

    Args:
        df (pl.DataFrame): DataFrame original.
        categorical_cols (List[str]): Columnas categóricas a codificar.

    Returns:
        Tuple[pl.DataFrame, List[str]]: DataFrame codificado y nombres de las columnas resultantes.
    """
    df_cat = df.select(categorical_cols)
    df_encoded = df_cat.to_dummies()
    df_non_cat = df.drop(categorical_cols)
    final_df = df_non_cat.hstack(df_encoded)
    return final_df, final_df.columns


def split_data(X: np.ndarray, y: np.ndarray, test_size: float = 0.2, random_state: int = 42) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Divide los datos en conjuntos de entrenamiento y prueba.

    Args:
        X (np.ndarray): Variables independientes.
        y (np.ndarray): Variable objetivo.
        test_size (float, optional): Proporción del conjunto de prueba. Default es 0.2.
        random_state (int, optional): Semilla para reproducibilidad. Default es 42.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: Conjuntos de entrenamiento y prueba para X e y.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

def train_and_evaluate_model(model, X_train: np.ndarray, X_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> float:
    """
    Entrena y evalúa un modelo de clasificación.

    Args:
        model: Modelo de clasificación.
        X_train (np.ndarray): Características de entrenamiento.
        X_test (np.ndarray): Características de prueba.
        y_train (np.ndarray): Etiquetas de entrenamiento.
        y_test (np.ndarray): Etiquetas de prueba.

    Returns:
        float: Precisión del modelo en el conjunto de prueba.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    return acc

Overwriting utils.py


## IMPLEMENTACIÓN PRINCIPAL

In [6]:
# LAB12-RUELAS.ipynb 

from utils import load_data, check_missing_values, remove_outliers, encode_categorical, split_data, train_and_evaluate_model
import polars as pl
import numpy as np
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Parte A: Preprocesamiento de Datos
print("Cargando datos...")
df = load_data(DATA_PATH)

print("Verificando valores faltantes...")
missing = check_missing_values(df)
print(missing)

print("Eliminando outliers...")
numerical_cols = ['Age', 'Height', 'Weight']
df_clean = remove_outliers(df, numerical_cols)

Cargando datos...
Verificando valores faltantes...
shape: (1, 17)
┌────────┬─────┬────────┬────────┬───┬─────┬──────┬────────┬────────────┐
│ Gender ┆ Age ┆ Height ┆ Weight ┆ … ┆ TUE ┆ CALC ┆ MTRANS ┆ NObeyesdad │
│ ---    ┆ --- ┆ ---    ┆ ---    ┆   ┆ --- ┆ ---  ┆ ---    ┆ ---        │
│ u32    ┆ u32 ┆ u32    ┆ u32    ┆   ┆ u32 ┆ u32  ┆ u32    ┆ u32        │
╞════════╪═════╪════════╪════════╪═══╪═════╪══════╪════════╪════════════╡
│ 0      ┆ 0   ┆ 0      ┆ 0      ┆ … ┆ 0   ┆ 0    ┆ 0      ┆ 0          │
└────────┴─────┴────────┴────────┴───┴─────┴──────┴────────┴────────────┘
Eliminando outliers...


In [7]:
# Parte B: Codificación de Variables Categóricas
print("Codificando variables categóricas...")
categorical_cols = [col for col in df_clean.columns if col not in numerical_cols + ['NObeyesdad']]
df_encoded, feature_names = encode_categorical(df_clean, categorical_cols)

# Preparar características y etiquetas
X = df_encoded.drop("NObeyesdad").to_numpy()
y = df_encoded["NObeyesdad"].to_numpy()

Codificando variables categóricas...


In [8]:
# Parte C: División de Datos y Modelado
print("Dividiendo datos en entrenamiento y prueba...")
X_train, X_test, y_train, y_test = split_data(X, y)

print("Entrenando y evaluando GaussianNB...")
gnb = GaussianNB()
acc_gnb = train_and_evaluate_model(gnb, X_train, X_test, y_train, y_test)

print("Entrenando y evaluando MultinomialNB...")
mnb = MultinomialNB()
acc_mnb = train_and_evaluate_model(mnb, X_train, X_test, y_train, y_test)

# Comparación de modelos
print(f"Precisión GaussianNB: {acc_gnb:.4f}")
print(f"Precisión MultinomialNB: {acc_mnb:.4f}")

Dividiendo datos en entrenamiento y prueba...
Entrenando y evaluando GaussianNB...
Accuracy: 0.5243
Classification Report:
                     precision    recall  f1-score   support

Insufficient_Weight       0.51      0.78      0.62        54
      Normal_Weight       0.50      0.18      0.26        56
     Obesity_Type_I       0.42      0.35      0.38        57
    Obesity_Type_II       0.39      0.94      0.55        54
   Obesity_Type_III       0.97      0.98      0.98        65
 Overweight_Level_I       0.38      0.15      0.21        54
Overweight_Level_II       0.45      0.20      0.27        51

           accuracy                           0.52       391
          macro avg       0.52      0.51      0.47       391
       weighted avg       0.53      0.52      0.48       391

Entrenando y evaluando MultinomialNB...
Accuracy: 0.6317
Classification Report:
                     precision    recall  f1-score   support

Insufficient_Weight       0.77      0.67      0.71        54


## TESTING (..test/test_utils.py)

In [ ]:
%%writefile ../tests/test_utils.py
# tests
import pytest
import polars as pl
import numpy as np
from sklearn.naive_bayes import GaussianNB
from src.utils import (
    load_data,
    check_missing_values,
    remove_outliers,
    encode_categorical,
    split_data,
    train_and_evaluate_model
)

def test_load_data(tmp_path):
    csv_path = tmp_path / "test.csv"
    csv_path.write_text("a,b\n1,2\n3,4")
    df = load_data(str(csv_path))
    assert df.shape == (2, 2)
    assert df.columns == ["a", "b"]

def test_check_missing_values():
    df = pl.DataFrame({"a": [1, None, 3], "b": [4, 5, None]})
    result = check_missing_values(df)

    # Convertimos a diccionario para comparar más fácilmente
    missing_dict = dict(zip(result.columns, result.row(0)))
    
    assert missing_dict["a"] == 1
    assert missing_dict["b"] == 1


def test_remove_outliers():
    df = pl.DataFrame({"a": [1, 2, 3, 100]})
    cleaned = remove_outliers(df, ["a"])
    assert 100 not in cleaned["a"].to_list()
    assert len(cleaned) == 3

def test_encode_categorical():
    df = pl.DataFrame({"color": ["red", "blue", "red"], "size": ["S", "M", "L"], "value": [10, 20, 30]})
    encoded_df, feature_names = encode_categorical(df, ["color", "size"])
    assert "value" in feature_names
    assert any("color_red" in col or "color_blue" in col for col in feature_names)
    assert encoded_df.shape[0] == 3

def test_split_data():
    X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
    y = np.array([0, 0, 1, 1])
    X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.5, random_state=0)
    assert len(X_train) == 2
    assert len(X_test) == 2
    assert sorted(np.concatenate([y_train, y_test])) == [0, 0, 1, 1]

def test_train_and_evaluate_model(capsys):
    X = np.array([[1, 2], [1, 3], [4, 5], [6, 7]])
    y = np.array([0, 0, 1, 1])
    X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.5, random_state=0)
    acc = train_and_evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test)
    captured = capsys.readouterr()
    assert "Accuracy:" in captured.out
    assert 0 <= acc <= 1

## EJECUCIÓN

In [10]:
# %pip install pytest

In [3]:
import os
import sys

# Ruta raíz de tu proyecto
project_root_path = r'C:\Ruelas\lab12\Data_Mining-LAB12'

# Cambiar el directorio de trabajo a la raíz del proyecto
%cd {project_root_path}
print(f"Directorio de trabajo actual establecido en: {os.getcwd()}")

# Configurar PYTHONPATH para que Python encuentre el paquete 'src'
os.environ['PYTHONPATH'] = project_root_path + os.pathsep + os.environ.get('PYTHONPATH', '')
print(f"PYTHONPATH establecido en: {os.environ['PYTHONPATH']}")

C:\Ruelas\lab12\Data_Mining-LAB12
Directorio de trabajo actual establecido en: C:\Ruelas\lab12\Data_Mining-LAB12
PYTHONPATH establecido en: C:\Ruelas\lab12\Data_Mining-LAB12;


In [6]:
!pytest tests/test_utils.py -v

============================= test session starts =============================
platform win32 -- Python 3.12.0, pytest-8.4.0, pluggy-1.6.0 -- c:\Ruelas\lab12\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Ruelas\lab12\Data_Mining-LAB12
configfile: pytest.ini
collecting ... collected 6 items

tests/test_utils.py::test_load_data PASSED                               [ 16%]
tests/test_utils.py::test_check_missing_values PASSED                    [ 33%]
tests/test_utils.py::test_remove_outliers PASSED                         [ 50%]
tests/test_utils.py::test_encode_categorical PASSED                      [ 66%]
tests/test_utils.py::test_split_data PASSED                              [ 83%]
tests/test_utils.py::test_train_and_evaluate_model PASSED                [100%]

============================== 6 passed in 1.42s ==============================
